# Building a Local Agent with Strands Agents and Ollama Model

This notebook demonstrates how to create a personal agent using Strands Agent and Ollama. The agent will be capable of performing various local tasks such as file operations, web searches, and system commands.

## What is Ollama?

[Ollama](https://ollama.com/) is an open-source framework that allows you to run large language models (LLMs) locally on your machine. It provides a simple API for interacting with these models, making it ideal for privacy-focused applications where you don't want to send data to external services.

Key benefits of Ollama:
- **Privacy**: All processing happens locally on your machine
- **No API costs**: Free to use as much as you want
- **Offline capability**: Works without internet connection
- **Customization**: Can be fine-tuned for specific use 


## Agent Details

<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Feature used        |Ollama Model - to create a file operations agent   |
|Agent Structure     |Single agent architecture                          |


</div>


### Agent Architecture

<div style="text-align:center">
    <img src="images/architecture.png" width="65%" />
</div>

In [1]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [ollama]


## Setup and Installation

Before running this notebook, make sure you have:

1. Install Ollama: Follow instructions at [https://ollama.com/download](https://ollama.com/download)
2. Start the Ollama server: `ollama serve`
3. Downloaded a model with Ollama: `ollama pull llama3.2:1b`

Refer to the [documentation](https://cuddly-sniffle-lrmk2y7.pages.github.io/0.1.x/user-guide/concepts/model-providers/ollama/) for detailed instructions.

In this notebook, we will download Ollama for the linux distribution for compatibility with SageMaker Studio. This is done for code execution during AWS lead workshops on Workshop Studio. If you are running this code locally, you should adjust the steps to download Ollama to your current enviroment.

In [2]:
# this will work on linux computers
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
import subprocess
subprocess.Popen(['ollama', 'serve'])

<Popen: returncode: None args: ['ollama', 'serve']>

Couldn't find '/home/sagemaker-user/.ollama/id_ed25519'. Generating new private key.
Your new public key is: 

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIKev77QEGhCaHxdL4Zt1mbQCMcLp2lvSyrCKalrprW+c



time=2025-11-28T09:03:38.310Z level=INFO source=routes.go:1544 msg="server config" env="map[CUDA_VISIBLE_DEVICES: GGML_VK_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:4096 OLLAMA_DEBUG:INFO OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/home/sagemaker-user/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://* vscode-webview://* vscode-file://*] OLLAMA_REMOTES:[ollama.com] 

In [9]:
!ollama pull llama3.2:latest

[GIN] 2025/11/28 - 09:10:21 | 200 |      26.988µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ [GIN] 2025/11/28 - 09:10:21 | 200 |  433.347447ms |       127.0.0.1 | POST     "/api/pull"
pulling manifest ⠸ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [5]:
# Import necessary libraries
import os

import requests

# Import strands components
from strands import Agent, tool
from strands.models.ollama import OllamaModel

In [11]:
# Check if Ollama is running:
try:
    response = requests.get("http://localhost:11434/api/tags")
    print("✅ Ollama is running. Available models:")
    for model in response.json().get("models", []):
        print(f"- {model['name']}")
except requests.exceptions.ConnectionError:
    print("❌ Ollama is not running. Please start Ollama before proceeding.")

[GIN] 2025/11/28 - 09:11:02 | 200 |     779.219µs |       127.0.0.1 | GET      "/api/tags"
✅ Ollama is running. Available models:
- llama3.2:latest
- llama3.2:3b


In [7]:
from strands.models.ollama import OllamaModel

# Configure the Ollama model
ollama_model = OllamaModel(
    model_id="llama3.2:latest", # Make sure this is the model you downloaded from Ollama
    host="http://localhost:11434",
    params={
        "max_tokens": 2048,  # Adjust based on your model's capabilities
        "temperature": 0.7,  # Lower for more deterministic responses, higher for more creative
        "top_p": 0.9,        # Nucleus sampling parameter
        "stream": True,      # Enable streaming responses
    },
)

## Defining Custom Tools

Tools are functions that the agent can use to interact with the environment. Below, we define several useful tools for our personal agent.

In [14]:
# File Operation Tools
@tool
def file_read(file_path: str) -> str:
    """Read a file and return its content. Supports both text and PDF files.

    Args:
        file_path (str): Path to the file to read

    Returns:
        str: Content of the file

    Raises:
        FileNotFoundError: If the file doesn't exist
    """
    try:
        # Check if it's a PDF file
        if file_path.lower().endswith('.pdf'):
            import PyPDF2
            with open(file_path, "rb") as file:
                pdf_reader = PyPDF2.PdfReader(file)
                text = ""
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
                return text if text.strip() else "Error: Could not extract text from PDF"
        else:
            # Regular text file
            with open(file_path, "r", encoding="utf-8") as file:
                return file.read()
    except FileNotFoundError:
        return f"Error: File '{file_path}' not found."
    except Exception as e:
        return f"Error reading file: {str(e)}"


@tool
def file_write(file_path: str, content: str) -> str:
    """Write content to a file.

    Args:
        file_path (str): The path to the file
        content (str): The content to write to the file

    Returns:
        str: A message indicating success or failure
    """
    try:
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(os.path.abspath(file_path)), exist_ok=True)

        with open(file_path, "w") as file:
            file.write(content)
        return f"File '{file_path}' written successfully."
    except Exception as e:
        return f"Error writing to file: {str(e)}"


@tool
def list_directory(directory_path: str = ".") -> str:
    """List files and directories in the specified path.

    Args:
        directory_path (str): Path to the directory to list

    Returns:
        str: A formatted string listing all files and directories
    """
    try:
        items = os.listdir(directory_path)
        files = []
        directories = []

        for item in items:
            full_path = os.path.join(directory_path, item)
            if os.path.isdir(full_path):
                directories.append(f"Folder: {item}/")
            else:
                files.append(f"File: {item}")

        result = f"Contents of {os.path.abspath(directory_path)}:\n"
        result += (
            "\nDirectories:\n" + "\n".join(sorted(directories))
            if directories
            else "\nNo directories found."
        )
        result += (
            "\n\nFiles:\n" + "\n".join(sorted(files)) if files else "\nNo files found."
        )

        return result
    except Exception as e:
        return f"Error listing directory: {str(e)}"

## Creating the Ollama-powered Agent

Now we'll create our agent using the Ollama model and the tools we defined above.

Note: You can add more tools like `execute_commands`, `search_files` etc.

In [16]:
# Define a comprehensive system prompt for our agent
system_prompt = """
You are a helpful personal assistant capable of performing local file actions and simple tasks for the user.

Your key capabilities:
1. Read, understand, and summarize files.
2. Create and write to files.
3. List directory contents and provide information on the files.
4. Summarize text content

When using tools:
- Always verify file paths before operations
- Be careful with system commands
- Provide clear explanations of what you're doing
- If a task cannot be completed, explain why and suggest alternatives

Always be helpful, concise, and focus on addressing the user's needs efficiently.
"""

model_id = (
    "llama3.2:3b"  # You can change this to any model you have pulled with Ollama.
)

In [18]:
# Create the agent with tools
local_agent = Agent(
    system_prompt=system_prompt, # Define a system Prompt
    model=ollama_model,
    tools=[file_read, file_write, list_directory],  # Add your custom tools here
)

### Testing the Agent

Let's test our agent with some example tasks.

In [19]:
local_agent("Show me the files in the current directory")

time=2025-11-28T09:15:07.623Z level=WARN source=cpu_linux.go:130 msg="failed to parse CPU allowed micro secs" error="strconv.ParseInt: parsing \"max\": invalid syntax"
llama_model_loader: loaded meta data with 30 key-value pairs and 255 tensors from /home/sagemaker-user/.ollama/models/blobs/sha256-dde5aa3fc5ffc17176b5e8bdc82f587b24b2678c6c66101bf7da77af9f7ccdff (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 3B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Llama-3.


Tool #1: list_directory
[GIN] 2025/11/28 - 09:15:35 | 200 | 28.143890485s |       127.0.0.1 | POST     "/api/chat"
The current directory contains the following files and directories:

* A single Python notebook file named `ollama_file_ops_agent.ipynb`
* A text file named `requirements.txt` containing dependencies
* An `images` directory, possibly for storing images used in the model
* A `.ipynb_checkpoints` directory, which likely stores checkpoints for the Jupyter Notebook 

Would you like to read or write any of these files? Or perhaps perform an operation on one of the directories?[GIN] 2025/11/28 - 09:15:50 | 200 | 15.010556706s |       127.0.0.1 | POST     "/api/chat"


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'The current directory contains the following files and directories:\n\n* A single Python notebook file named `ollama_file_ops_agent.ipynb`\n* A text file named `requirements.txt` containing dependencies\n* An `images` directory, possibly for storing images used in the model\n* A `.ipynb_checkpoints` directory, which likely stores checkpoints for the Jupyter Notebook \n\nWould you like to read or write any of these files? Or perhaps perform an operation on one of the directories?'}]}, metrics=EventLoopMetrics(cycle_count=2, tool_metrics={'list_directory': ToolMetrics(tool={'toolUseId': 'list_directory', 'name': 'list_directory', 'input': {'directory_path': '.'}}, call_count=1, success_count=1, error_count=0, total_time=0.0005338191986083984)}, cycle_durations=[15.012617826461792], traces=[<strands.telemetry.metrics.Trace object at 0x7f34b844d1f0>, <strands.telemetry.metrics.Trace object at 0x7f34ba17

In [21]:
local_agent("Create a file called 'sample.txt' with the content 'This is a test file created by my Ollama agent.'")


Tool #2: file_write

Tool #3: file_write
[GIN] 2025/11/28 - 09:16:53 | 200 | 24.783465127s |       127.0.0.1 | POST     "/api/chat"
The file `sample.txt` has been created in the current directory with the specified content. The file now contains:

```
This is a test file created by my Ollama agent.
```[GIN] 2025/11/28 - 09:17:03 | 200 |  9.558477298s |       127.0.0.1 | POST     "/api/chat"


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'The file `sample.txt` has been created in the current directory with the specified content. The file now contains:\n\n```\nThis is a test file created by my Ollama agent.\n```'}]}, metrics=EventLoopMetrics(cycle_count=5, tool_metrics={'list_directory': ToolMetrics(tool={'toolUseId': 'list_directory', 'name': 'list_directory', 'input': {'directory_path': '.'}}, call_count=1, success_count=1, error_count=0, total_time=0.0005338191986083984), 'file_write': ToolMetrics(tool={'toolUseId': 'file_write', 'name': 'file_write', 'input': {'content': 'This is a test file created by my Ollama agent.', 'file_path': 'sample.txt'}}, call_count=2, success_count=2, error_count=0, total_time=0.0023565292358398438)}, cycle_durations=[15.012617826461792, 9.560912609100342], traces=[<strands.telemetry.metrics.Trace object at 0x7f34b844d1f0>, <strands.telemetry.metrics.Trace object at 0x7f34ba170170>, <strands.telemetry.

In [22]:
!cat sample.txt

This is a test file created by my Ollama agent.

In [24]:
## Trying with some unknown file
local_agent("Read the file 'document.txt' and summarize it in 5 bullet points.")


Tool #4: file_read
[GIN] 2025/11/28 - 09:19:55 | 200 | 20.085597099s |       127.0.0.1 | POST     "/api/chat"
It appears that there is no file named 'document.txt' in the current directory.

Would you like to create a new file with this name, or perhaps specify a different file path? I can help you with either of these options.[GIN] 2025/11/28 - 09:20:04 | 200 |  8.802161933s |       127.0.0.1 | POST     "/api/chat"


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "It appears that there is no file named 'document.txt' in the current directory.\n\nWould you like to create a new file with this name, or perhaps specify a different file path? I can help you with either of these options."}]}, metrics=EventLoopMetrics(cycle_count=8, tool_metrics={'list_directory': ToolMetrics(tool={'toolUseId': 'list_directory', 'name': 'list_directory', 'input': {'directory_path': '.'}}, call_count=1, success_count=1, error_count=0, total_time=0.0005338191986083984), 'file_write': ToolMetrics(tool={'toolUseId': 'file_write', 'name': 'file_write', 'input': {'content': 'This is a test file created by my Ollama agent.', 'file_path': 'sample.txt'}}, call_count=2, success_count=2, error_count=0, total_time=0.0023565292358398438), 'file_read': ToolMetrics(tool={'toolUseId': 'file_read', 'name': 'file_read', 'input': {'file_path': 'document.txt'}}, call_count=1, success_count=1, error_cou

In [26]:
## Trying with sample.txt file
local_agent("Read the file 'sample.txt' and summarize it in 5 bullet points.")


Tool #5: file_read
[GIN] 2025/11/28 - 09:21:00 | 200 | 20.336074956s |       127.0.0.1 | POST     "/api/chat"
Here are five bullet points summarizing the content of 'sample.txt':

* The file 'sample.txt' was created by an Ollama agent.
* The purpose of the file appears to be demonstration or testing.
* No specific information about the user is provided.
* The file does not contain any additional context or details.
* It serves as a starting point for further development or exploration.[GIN] 2025/11/28 - 09:21:13 | 200 | 12.150845809s |       127.0.0.1 | POST     "/api/chat"


AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Here are five bullet points summarizing the content of 'sample.txt':\n\n* The file 'sample.txt' was created by an Ollama agent.\n* The purpose of the file appears to be demonstration or testing.\n* No specific information about the user is provided.\n* The file does not contain any additional context or details.\n* It serves as a starting point for further development or exploration."}]}, metrics=EventLoopMetrics(cycle_count=11, tool_metrics={'list_directory': ToolMetrics(tool={'toolUseId': 'list_directory', 'name': 'list_directory', 'input': {'directory_path': '.'}}, call_count=1, success_count=1, error_count=0, total_time=0.0005338191986083984), 'file_write': ToolMetrics(tool={'toolUseId': 'file_write', 'name': 'file_write', 'input': {'content': 'This is a test file created by my Ollama agent.', 'file_path': 'sample.txt'}}, call_count=2, success_count=2, error_count=0, total_time=0.002356529235839

In [28]:
local_agent(
    "Read the file in the path `sample_file/Amazon-com-Inc-2023-Shareholder-Letter.pdf` and summarize it in 5 bullet points."
)


Tool #6: file_read
[GIN] 2025/11/28 - 09:24:12 | 200 |  23.56660999s |       127.0.0.1 | POST     "/api/chat"


time=2025-11-28T09:24:12.714Z level=WARN source=runner.go:152 msg="truncating input prompt" limit=4096 prompt=9380 keep=5 new=4096


Andy Jassy, the current CEO of Amazon, provided an overview of the company's past, present, and future in his 2021 letter to Amazon shareholders. Here's a summary:

**Historical Context**: The letter begins by referencing Amazon's humble beginnings as an online bookstore. In 1997, Jeff Bezos had just launched the company and was facing challenges from established players in the retail industry.

**Key Milestones**:

* By year-end 1997, Amazon had served over 1.5 million customers, with revenue growth of 838% to $147.8 million.
* The company's employee base grew from 158 to 614, and its distribution center capacity expanded to 285,000 square feet.

**Focus on Customers**: Bezos emphasized the importance of customer satisfaction, citing the company's focus on providing a better shopping experience through features like gift certificates, 1-Click shopping, and vast product selection. He noted that word-of-mouth referrals remained a key driver of customer acquisition.

**Strategic Prioriti

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': "Andy Jassy, the current CEO of Amazon, provided an overview of the company's past, present, and future in his 2021 letter to Amazon shareholders. Here's a summary:\n\n**Historical Context**: The letter begins by referencing Amazon's humble beginnings as an online bookstore. In 1997, Jeff Bezos had just launched the company and was facing challenges from established players in the retail industry.\n\n**Key Milestones**:\n\n* By year-end 1997, Amazon had served over 1.5 million customers, with revenue growth of 838% to $147.8 million.\n* The company's employee base grew from 158 to 614, and its distribution center capacity expanded to 285,000 square feet.\n\n**Focus on Customers**: Bezos emphasized the importance of customer satisfaction, citing the company's focus on providing a better shopping experience through features like gift certificates, 1-Click shopping, and vast product selection. He noted 

In [30]:
# Example 4: create a readme file after reading and understanding multiple files
response = local_agent("Create a readme.md for the current directory")


Tool #7: file_write

Tool #8: file_write
[GIN] 2025/11/28 - 09:31:02 | 200 | 52.648088741s |       127.0.0.1 | POST     "/api/chat"
Here is the complete README.md file for the current directory:

```
# README
## Description
This is the root directory of our project.

## Getting Started
To get started, navigate to this directory and run `npm install`.
```[GIN] 2025/11/28 - 09:31:12 | 200 | 10.418170696s |       127.0.0.1 | POST     "/api/chat"


### Lets try to configure Web Search 

In [36]:
system_prompt = """
You are a helpful personal assistant capable of performing web search for technology related updates, local file actions and simple tasks for the user.

Your key capabilities:
1. Read, understand, and summarize files.
2. Create and write to files.
3. List directory contents and provide information on the files.
4. Summarize text content
5. Sacn websites and mention sources of information

When using tools:
- Always verify file paths before operations
- Be careful with system commands
- Provide clear explanations of what you're doing
- If a task cannot be completed, explain why and suggest alternatives

Always be helpful, concise, and focus on addressing the user's needs efficiently.
"""

model_id = (
    "llama3.2:3b"  # You can change this to any model you have pulled with Ollama.
)

In [37]:
# Create the agent with tools

from ollama import web_fetch, web_search

local_agent = Agent(
    system_prompt=system_prompt, # Define a system Prompt
    model=ollama_model,
    tools=[file_read, file_write, list_directory, web_search],  # Add your custom tools here
)

tool=<<bound method Client.web_search of <ollama._client.Client object at 0x7f34b9701ac0>>> | unrecognized tool specification


In [39]:
local_agent(
    "Whats are the latest developments in AWS for Agentic AI"
)

time=2025-11-28T09:39:32.716Z level=WARN source=cpu_linux.go:130 msg="failed to parse CPU allowed micro secs" error="strconv.ParseInt: parsing \"max\": invalid syntax"
llama_model_loader: loaded meta data with 30 key-value pairs and 255 tensors from /home/sagemaker-user/.ollama/models/blobs/sha256-dde5aa3fc5ffc17176b5e8bdc82f587b24b2678c6c66101bf7da77af9f7ccdff (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 3B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Llama-3.


Tool #1: file_read
[GIN] 2025/11/28 - 09:40:04 | 200 | 32.160558467s |       127.0.0.1 | POST     "/api/chat"
It seems that I was unable to access the requested file.

However, I can suggest an alternative approach. AWS has made significant developments in Agent-based AI (also known as Multi-agent Systems or MAS) through its various services and offerings. Here are some of the latest updates:

1. **Amazon SageMaker Autopilot**: This is a fully managed service that enables businesses to build, deploy, and manage multi-agent systems using pre-built templates and APIs.
2. **AWS DeepRacer**: This is a cloud-based autonomous racing platform that uses agent-based AI to enable developers to build and train their own autonomous vehicle models.
3. **Amazon Rekognition Edge**: This service allows developers to use pre-trained computer vision models, including those for object detection and face recognition, which can be used in multi-agent systems.
4. **AWS IoT Core**: This is a managed cloud s

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'It seems that I was unable to access the requested file.\n\nHowever, I can suggest an alternative approach. AWS has made significant developments in Agent-based AI (also known as Multi-agent Systems or MAS) through its various services and offerings. Here are some of the latest updates:\n\n1. **Amazon SageMaker Autopilot**: This is a fully managed service that enables businesses to build, deploy, and manage multi-agent systems using pre-built templates and APIs.\n2. **AWS DeepRacer**: This is a cloud-based autonomous racing platform that uses agent-based AI to enable developers to build and train their own autonomous vehicle models.\n3. **Amazon Rekognition Edge**: This service allows developers to use pre-trained computer vision models, including those for object detection and face recognition, which can be used in multi-agent systems.\n4. **AWS IoT Core**: This is a managed cloud service that enab

## Conclusion

In this notebook, we've created a local personal agent using Stands and Ollama. The agent can perform file operations (read, write, append) and Summarize/Analyze text

This demonstrates the power of running AI models locally with Ollama, combined with the flexibility of strands' tool system. You can extend this agent by adding more tools or using different Ollama models based on your needs.

### Next Steps (Ideas)

- Try different Ollama models to see how they affect the agent's capabilities
- Add more complex tools like web search, email sending, or calendar integration
- Implement memory for the agent to remember past interactions
- Create a simple UI for interacting with your agent